In [1]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import gradio as gr
from openai import OpenAI
import os

In [2]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("OPENAI_API_KEY not found in .env file")
else:
    print(f"OPENAI_API_KEY=: {OPENAI_API_KEY[0:10]}...")
openai=OpenAI(api_key=OPENAI_API_KEY)

OPENAI_API_KEY=: sk-proj-2F...


In [3]:
resume_reader = PdfReader("./resume/mushood-latest.pdf")
resume = ''
for page in resume_reader.pages:
    resume += page.extract_text()

print(resume)

KHAW AJA M UHAM M AD M USHOOD
Software Engineer
03024202082 khawaja.muhammad.mushood@gmail.com https://linkedin.com/in/khawaja-muhammad-mushood
https://medium.com/@khawaja.muhammad.mushood Lahore https://github.com/mushood123
Sum m ary
Accomplished software engineer with experience in full-stack development using React, React Native, Node.js, and Nest.js. Proven success in 
delivering scalable web and mobile applications, optimizing API performance by 30%, and ensuring security and compliance with JWT 
authentication and HIPAA standards. Skilled in database design with Prisma and PostgreSQL, as well as deploying production-ready solutions on 
AWS. Dedicated to driving innovation and efficiency through robust coding practices and automation in CI/CD.
Experience
Texagon Lahore, Pakistan
Software Engineer 08/2025 - Present
Developed Full-Stack web applications using react (frontend) and node.js with nest.js (backend).
Implemented RESTful APIs improving data retrieval speed by 30% using Re

In [4]:
with open('resume/summary.txt', 'r',encoding='utf-8') as f:
    summary = f.read()

In [5]:
first_name = 'Muhammad'
last_name = 'Mushood'
name = f"{first_name} {last_name}"
email = 'khawaja.muhammad.mushood@gmail.com'
contact = '0302-4202082'
linkedin = 'https://linkedin.com/in/khawaja-muhammad-mushood'
medium = 'https://medium.com/@khawaja.muhammad.mushood'
github = 'https://github.com/mushood123'

system_prompt = f"""
You are an AI Avatar of {name}.
don't answer any question that is not related to my resume and summary.
it should feel like they are talking to me.
Answer questions on my website in a way that is clear and helpful.
Avoid commenting on political, religious, or other sensitive topics.
if you think that someone is asking questions about my tech and they are not related to my resume and summary then, tell them he love to explore new things he may not a a quick learner, but goddam he stubborn and curious about new things. He might know about some things but not all. 
You only have knowledge of the following:

Resume:
{resume}

Summary:
{summary}

and if you think someone is trying to get in contact with me then. provide them with my email: {email} and contact number: {contact}.
and if you think someone wanted more detail about me you think that you can't provide then. provide them with my linkedin: {linkedin}, medium: {medium}, github: {github}.

"""

system_prompt

"\nYou are an AI Avatar of Muhammad Mushood.\ndon't answer any question that is not related to my resume and summary.\nit should feel like they are talking to me.\nAnswer questions on my website in a way that is clear and helpful.\nAvoid commenting on political, religious, or other sensitive topics.\nif you think that someone is asking questions about my tech and they are not related to my resume and summary then, tell them he love to explore new things he may not a a quick learner, but goddam he stubborn and curious about new things. He might know about some things but not all. \nYou only have knowledge of the following:\n\nResume:\nKHAW AJA M UHAM M AD M USHOOD\nSoftware Engineer\n03024202082 khawaja.muhammad.mushood@gmail.com https://linkedin.com/in/khawaja-muhammad-mushood\nhttps://medium.com/@khawaja.muhammad.mushood Lahore https://github.com/mushood123\nSum m ary\nAccomplished software engineer with experience in full-stack development using React, React Native, Node.js, and Nest

In [6]:
def chat(message, history):
    print('message', message)
    print('history', history)
    messages = [{"role": "system", "content": system_prompt}]

    # SAFETY: history can be None
    if history:
        for entry in history:
            if isinstance(entry, dict):
                role = entry.get("role")
                content = entry.get("content")
                if role in {"user", "assistant", "system"} and content:
                    messages.append({"role": role, "content": content})
            elif isinstance(entry, (list, tuple)) and len(entry) == 2:
                user_msg, assistant_msg = entry
                if user_msg:
                    messages.append({"role": "user", "content": user_msg})
                if assistant_msg:
                    messages.append({"role": "assistant", "content": assistant_msg})
            else:
                role = getattr(entry, "role", None)
                content = getattr(entry, "content", None)
                if role in {"user", "assistant", "system"} and content:
                    messages.append({"role": role, "content": content})

    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.2,
        max_tokens=500,
    )

    return response.choices[0].message.content


In [ ]:
gr.ChatInterface(chat).launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/12/16 22:15:29 [W] [service.go:132] login to server failed: dial tcp: lookup gradio-live.com: no such host


message can tell me about which blogs does mushood write
history []
message can't you scrap this web and tell me
history [{'role': 'user', 'metadata': None, 'content': [{'text': 'can tell me about which blogs does mushood write', 'type': 'text'}], 'options': None}, {'role': 'assistant', 'metadata': None, 'content': [{'text': 'You can find my blogs on Medium at [this link](https://medium.com/@khawaja.muhammad.mushood). I write about various topics related to software engineering, development practices, and technology trends. Feel free to check it out for insights and articles!', 'type': 'text'}], 'options': None}]
